In [69]:
import subprocess

infile="/mnt/disk1/tapearchive/A4/211008_0016.wav" 
outfile = "/home/caiwan/test3.flac"

filter_stack = ";".join([
    "acopy[out]",
    "[out]channelsplit=channel_layout=stereo:channels=FL[out]"
    "[out]dynaudnorm=framelen=1000:maxgain=3:coupling=false[out]"
])

ffmpeg_commnad=f"ffmpeg -y -i {infile} -filter_complex {filter_stack} -map [out] {outfile}"

print(ffmpeg_commnad.split())

ffmpeg_process = subprocess.Popen(ffmpeg_commnad.split(), stdout=subprocess.PIPE, stdin=subprocess.PIPE)

while ffmpeg_process.poll() is None:
    try:
        stdout_data, stderr_data = ffmpeg_process.communicate(timeout=30)
    except subprocess.TimeoutExpired:
        ffmpeg_process.kill()
        stdout_data, stderr_data = ffmpeg_process.communicate()
    print(stdout_data, stderr_data)

['ffmpeg', '-y', '-i', '/mnt/disk1/tapearchive/A4/211008_0016.wav', '-filter_complex', 'acopy[out];[out]dynaudnorm=framelen=1000:maxgain=3:coupling=false[out]', '-map', '[out]', '/home/caiwan/test3.flac']


ffmpeg version n4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.1.0 (GCC)
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libmfx --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librav1e --enable-librsvg --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxcb --enable-libxml2 --enable-libxvid --e

b'' None


In [1]:
import sys
import logging


LOGGER = logging.getLogger()
LOGGER.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stderr)

formatter = logging.Formatter('%(threadName)s: %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

LOGGER.handlers = [handler]

In [3]:
import waiting
from contextlib import ExitStack

from tq.task_dispacher import LocalTaskQueue, TaskDispatcher
from tq.job_system import JobManager

from tapearchive.models.records import ChannelMode
from tapearchive.workflow.audio_convert import ConvertAudio, AudioConverterHandler

# Do task
task_queue = LocalTaskQueue()
with ExitStack() as stack:
    job_manager = stack.enter_context(JobManager())
    dispatcher = stack.enter_context(TaskDispatcher(task_queue, job_manager))

    audio_converter_handler = AudioConverterHandler()
    dispatcher.register_task_handler(audio_converter_handler)

    logging.debug(dispatcher.task_handlers)

    dispatcher.post_task(ConvertAudio(
        source="/mnt/disk1/tapearchive/B4/211014_0050.wav",
        source_channel=ChannelMode.LEFT,
        target = "/home/caiwan/test_L.flac",
    ))

    dispatcher.post_task(ConvertAudio(
        source="/mnt/disk1/tapearchive/B4/211014_0050.wav",
        source_channel=ChannelMode.RIGHT,
        target = "/home/caiwan/test_R.flac",
    ))

    waiting.wait(lambda: len(audio_converter_handler.tasks_finished)==2, sleep_seconds=1, timeout_seconds=120)

    logging.info(audio_converter_handler.tasks_finished)

MainThread: root - DEBUG - {<class 'tapearchive.workflow.audio_convert.ConvertAudio'>: [<function AudioConverterHandler.convert_audio at 0x7f0a81856040>], <class 'tapearchive.workflow.audio_convert.AudioConversionDone'>: [<function AudioConverterHandler.conversion_done at 0x7f0a81844d30>], <class 'tapearchive.workflow.audio_convert.AudioConversionError'>: [<function AudioConverterHandler.conversion_error at 0x7f0a81844700>]}
MainThread: tq.task_dispacher - DEBUG - Task posted: ConvertAudio(source='/mnt/disk1/tapearchive/B4/211014_0050.wav', source_channel=<ChannelMode.LEFT: 'left'>, target='/home/caiwan/test_L.flac')
MainThread: tq.task_dispacher - DEBUG - Task posted: ConvertAudio(source='/mnt/disk1/tapearchive/B4/211014_0050.wav', source_channel=<ChannelMode.RIGHT: 'right'>, target='/home/caiwan/test_R.flac')
Thread-9: tq.task_dispacher - DEBUG - dispatch loop_tick
Thread-9: tq.task_dispacher - INFO - Dispatch task: ConvertAudio(source='/mnt/disk1/tapearchive/B4/211014_0050.wav', sou